In [40]:
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
from openai import OpenAI

In [44]:
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document

from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
documents = SimpleDirectoryReader(input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]).load_data()

In [45]:
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [42]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [ ]:
from trulens.dashboard import run_dashboard
run_dashboard(session)

Starting dashboard ...
Dashboard already running at path:   Local URL: http://localhost:52847



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [66]:
run_dashboard(session)

Starting dashboard ...
Dashboard already running at path:   Local URL: http://localhost:52847



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
# from trulens.providers.openai import OpenAI
# from trulens.core import Feedback
# provider = OpenAI()

In [ ]:
# from trulens.feedback.v2.feedback import Groundedness
# # Initialize Groundedness
# grounded = Groundedness()

In [ ]:
# from trulens.feedback.v2.feedback import Groundedness
# openai_provider = OpenAI(model_engine="gpt-3.5-turbo")
# grounded = Groundedness(groundedness_provider=openai_provider)

In [ ]:
# from trulens.feedback.v2.feedback import Groundedness
# from trulens.core import Feedback
# f_groundedness = (
#     Feedback(
#         openai_provider.groundedness_measure,
#         name="Groundedness OpenAI GPT-3.5"
#     )
#     .on_input()  # Source text
#     .on_output()  # Response text
#     .aggregate(groundedness.grounded_statements_aggregator)
# )

In [ ]:
#Imported from StackOverflow

import numpy as np
from trulens.apps.llamaindex import TruLlama
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.

context = TruLlama.select_context(query_engine)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

In [55]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document
from llama_index.core.node_parser import HierarchicalNodeParser

from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage
import os
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

In [53]:

from llama_index.llms.openai import OpenAI
llm = OpenAI()
embed_model = OpenAIEmbedding()


In [54]:
Settings.llm = llm
Settings.embed_model = embed_model

In [47]:
def build_sentence_window_index():
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    nodes = node_parser.get_nodes_from_documents(documents)
    sentence_index = VectorStoreIndex(nodes)

    return sentence_index

In [ ]:

# from llama_index.core import SimpleDirectoryReader

# documents = SimpleDirectoryReader(
#     input_files=["./IPCC_AR6_WGII_Chapter03.pdf"]
# ).load_data()

In [48]:
# node_parser = SentenceWindowNodeParser.from_defaults(
#     window_size=3,
#     window_metadata_key="window",
#     original_text_metadata_key="original_text",
# )
# nodes = node_parser.get_nodes_from_documents(documents)
# sentence_index = VectorStoreIndex(nodes)

sentence_index = build_sentence_window_index()



In [49]:
def get_sentence_window_query_engine(
    sentence_index,
    similarity_top_k=6,
    rerank_top_n=2,
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=rerank_top_n
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [65]:
response = sentence_window_engine.query("How do you create your AI portfolio?")
print(response)

Building a portfolio of projects in AI involves progressing from simple to complex undertakings over time. Communication is crucial in explaining your thought process and the value of your work to others. This helps in gaining trust and securing resources for larger projects. Additionally, showcasing leadership skills, even without a formal title, becomes essential when working on bigger AI projects that require teamwork.


In [59]:
## Reference documents
# https://www.trulens.org/getting_started/quickstarts/llama_index_quickstart/
import numpy as np
from trulens.apps.llamaindex import TruLlama
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

# Initialize provider class
provider = OpenAI(model_engine="gpt-4.1-mini")

# select context to be used in feedback. the location of context is app specific.

context = TruLlama.select_context(sentence_window_engine)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [60]:
# Initializing the app
tru_query_engine_recorder = TruLlama(
    sentence_window_engine,
    app_name="Deep_App",
    app_version="base",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'abc.ABC'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'

To create your AI portfolio, it is important to start with simple projects and gradually progress to more complex undertakings over time. Communication plays a crucial role in showcasing the value of your work and gaining the trust of others to invest in larger projects. Clearly explaining your thought process and accomplishments is essential. Additionally, developing leadership skills, even if you are not in a formal leadership position, is important for handling larger AI projects that require teamwork.


In [62]:
with tru_query_engine_recorder as recording:
    sentence_window_engine.query("How do you create your AI portfolio?")